# Install

In [ ]:
!python3 --version

In [ ]:
!pip3 install uv

In [ ]:
!uv pip install --system kara-kanji-sync

# Inputs

In [ ]:
# @title ASS File (upload)
# @markdown Run this cell to **upload your ass file** using the form below.
#
# @markdown **Note**: If the upload fails, try using a different browser or upload the file manually in the Files section on the left sidebar.

from google.colab import files

upload = files.upload()
for file in upload:
    song_filename = file
    break

In [ ]:
# @markdown **If you uploaded the file manually, enter the song filename here.**

song_filename_override = "" # @param {type:"string"}
song_filename = song_filename_override or song_filename

In [ ]:
# @title Video resolution
# @markdown Input the vertical resolution of the video (1080, 720, 480 etc)

resolution = 1080 # @param {type:"string"}

In [ ]:
# @title Lyrics input
# @markdown Run this cell to **paste the lyrics** using the cell below.
#
# @markdown Then ajust it to match those from ASS file. They must have the same number of lines.

from IPython.display import display
from IPython.core.display import HTML
from ipywidgets import Layout, Textarea, HBox
from kara_kanji_sync import KanjiSyncer
import pysubs2
import logging

logging.basicConfig(level=logging.INFO, force=True)
for handler in logging.root.handlers:
    handler.addFilter(logging.Filter('kara_kanji_sync'))

subs = pysubs2.load(song_filename)

kanji_syncer = KanjiSyncer()
kanji_syncer.subtitles_file = subs
a = kanji_syncer.make_subtitles_lines()
lines = "\n".join([line.plaintext for line in kanji_syncer.subtitle_lines])
height = len(kanji_syncer.subtitle_lines) * 20
line_numbers = "\n".join(str(i) for i in range(1, len(kanji_syncer.subtitle_lines) + 1))
lyrics_area = Textarea(layout=Layout(width="20em", height=f"{height}px"))
number_area = Textarea(layout=Layout(width="3em", height=f"{height}px"),
                       value=line_numbers, disabled=True)
subs_lyrics_output = Textarea(
    layout=Layout(width="40em", height=f"{height}px"),
    value=lines,
    disabled=True,
    style=dict(font_size='40px')
)
display(HTML("<style>textarea, input {font-family: Yu Gothic;}</style>"))
display(HBox([lyrics_area, number_area, subs_lyrics_output]))

In [ ]:
# @title Lyrics check
# @markdown Run this cell to have a preliminary check of the matching.

kanji_syncer.lyrics = lyrics_area.value.splitlines()
kanji_syncer.make_matches_and_groups()

non_matching_lines = [match_index for match_index, matches in enumerate(kanji_syncer.all_matches)
if not matches]
if not non_matching_lines:
  print("All lines seems to match :)")
else:
  for non_matches_index in non_matching_lines:
    print(f"Matching issue on line {str(non_matches_index + 1)}")
  print("You should modify your input above and rerun this cell to fix the matching issues.")

# Run

In [ ]:
# @title Generate
# @markdown Run this cell to generate the result.
kanji_syncer.resolution = resolution
kanji_file = kanji_syncer.sync_subs()


In [ ]:
# @title Save and download karaoke file
# @markdown Run this cell to download the file.
from pathlib import Path
from google.colab import files

song_path = Path(song_filename)

subs_path = song_path.with_suffix('.jpn.ass')
kanji_file.save(subs_path)
files.download(subs_path)

After downloading the ASS file, there are some steps to follow to display the karaoke correctly.


1.   Open the ASS file.
2.   Add the video.
3.   Click on "Automation" -> "Apply karaoke template"





